<a href="https://colab.research.google.com/github/sarnesh444/Coursera_Capstone/blob/master/week5capstonefinalproject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# IBM Applied Data Science with Capstone by Coursera

# Week5 final project 

# Predicting the best location for a business startups in a particular neighborhood

### Importing the necessary libraries

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

/bin/bash: conda: command not found
Libraries imported.


In [2]:
pip install beautifulsoup4

In [3]:
pip install lxml

In [4]:
pip install html5lib

In [5]:
pip install requests

In [0]:
from bs4 import BeautifulSoup
import requests

## After importing the required libraries, we get the data from Wikipedia website and format it into a table

In [0]:
source=requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M").text

In [0]:
soup=BeautifulSoup(source,'lxml')

In [0]:
table_soup=soup.table.tbody.find_all('tr')

In [0]:
table_soup.remove(table_soup[0])

In [0]:
pc=list()
br=list()
ne=list()
for row in table_soup:
    try:
        pc.append(row.find_all('td')[0].text)
        br.append(row.find_all('td')[1].text)
        ne.append(row.find_all('td')[2].text.strip())
    except Exception as e:
        pass
toronto_df=pd.DataFrame({'Postalcode':pc,'Borough':br,'Neighborhood':ne})

In [0]:
df= toronto_df

In [14]:
toronto_df.head()

,Postalcode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


## We drop the values of Borough which are 'Not assigned'

In [0]:
toronto_df=toronto_df.drop(toronto_df[toronto_df['Borough']=='Not assigned'].index)

In [16]:
toronto_df.head()

,Postalcode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor


In [0]:
toronto_df.loc[toronto_df['Neighborhood']=='Not assigned','Neighborhood']=toronto_df.loc[toronto_df['Neighborhood']=='Not assigned','Borough']

In [0]:
toronto_gr=toronto_df.groupby("Postalcode")

In [19]:
df=pd.DataFrame(columns=['Postalcode','Borough','Neighborhood'])
i=0
for grp in pd.unique(toronto_df["Postalcode"]):
    ni=list()
    grp_df=toronto_gr.get_group(grp)
    ni=grp_df['Neighborhood'].values
    #temp_df=pd.DataFrame({'Postalcode':[grp],'Borough':pd.unique(grp_df['Borough']),'Neighborhood':','.join(ni)})
    #print(temp_df)
    df.loc[i]=[grp,pd.unique(grp_df['Borough'])[0],','.join(ni)]
    i=i+1
df.head()

,Postalcode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,"Lawrence Heights,Lawrence Manor"
4,M7A,Queen's Park,Queen's Park


In [20]:
#shape of the table after pre-processing
df.shape

(103, 3)

# Using the CSV file to create the required dataframe

In [21]:
!wget -o 'toranto_geodata.csv' https://cocl.us/Geospatial_data
data=pd.read_csv('https://cocl.us/Geospatial_data')
data.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [22]:
lat_arr=list()
lon_arr=list()
for postal_code in df['Postalcode'].values:
    lat_arr.append(data.loc[data['Postal Code']==postal_code,'Latitude'].values[0])
    lon_arr.append(data.loc[data['Postal Code']==postal_code,'Longitude'].values[0])
df['Latitude']=lat_arr
df['Longitude']=lon_arr
df.head()

,Postalcode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636
3,M6A,North York,"Lawrence Heights,Lawrence Manor",43.718518,-79.464763
4,M7A,Queen's Park,Queen's Park,43.662301,-79.389494


# Exploring and clustering the neighborhoods in Toronto

In [23]:
pip install geocoder

     |████████████████████████████████| 102kB 3.3MB/s 


In [0]:
import folium

In [25]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[43.651070, -79.347015], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df['Latitude'], df['Longitude'], df['Borough'], df['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [0]:
CLIENT_ID = 'FK4FOYRCONHDU1B4QAIPH30FYVPNOKZIQ3FLDNLE4ZDKTZGA' # your Foursquare ID
CLIENT_SECRET = 'IXFQYUOKRPTABOZB0PAGMAPJ1JLHGIOVIAZMWTBPUL4VWWZT' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

In [27]:
print(df['Neighborhood'].values[0],df['Postalcode'].values[0])

Parkwoods M3A


In [28]:
print('latitude :',df['Latitude'].values[0])
print('longitude:',df['Longitude'].values[0])

latitude : 43.7532586
longitude: -79.3296565


## Now, let's get the top 100 venues that are in Parkwoods M3A within a radius of 2 Km

In [29]:
LIMIT=100
radius=2000
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    43.7532586, 
    -79.3296565, 
    radius, 
    LIMIT)

url

'https://api.foursquare.com/v2/venues/explore?&client_id=FK4FOYRCONHDU1B4QAIPH30FYVPNOKZIQ3FLDNLE4ZDKTZGA&client_secret=IXFQYUOKRPTABOZB0PAGMAPJ1JLHGIOVIAZMWTBPUL4VWWZT&v=20180605&ll=43.7532586,-79.3296565&radius=2000&limit=100'

In [0]:
import json # library to handle JSON files
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

In [0]:
results = requests.get(url).json()

In [0]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [33]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Allwyn's Bakery,Caribbean Restaurant,43.759840,-79.324719
1,Donalda Golf & Country Club,Golf Course,43.752816,-79.342741
2,Brookbanks Park,Park,43.751976,-79.332140
3,Graydon Hall Manor,Event Space,43.763923,-79.342961
4,LCBO,Liquor Store,43.757774,-79.314257


In [34]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

93 venues were returned by Foursquare.


In [35]:
res2=results['response']['groups'][0]['items']
for res in res2:
    print(res['venue']['name'])
    print("     at a distance of---> ",res['venue']['location']['distance'])
    print()
print()

Allwyn's Bakery
     at a distance of--->  833

Donalda Golf & Country Club
     at a distance of--->  1053

Brookbanks Park
     at a distance of--->  245

Graydon Hall Manor
     at a distance of--->  1597

LCBO
     at a distance of--->  1336

Darband Restaurant
     at a distance of--->  1530

Island Foods
     at a distance of--->  1553

LA Fitness
     at a distance of--->  1532

Galleria Supermarket
     at a distance of--->  1597

North Beach Indoor Volleyball Academy
     at a distance of--->  1851

Tim Hortons
     at a distance of--->  866

Me Va Me Kitchen Express
     at a distance of--->  1798

The Captain's Boil
     at a distance of--->  1609

Matsuda Japanese Cuisine & Teppanyaki
     at a distance of--->  1561

A&W Canada
     at a distance of--->  852

Gilaneh
     at a distance of--->  1639

Baretto Caffé
     at a distance of--->  1669

Menchie's Frozen Yogurt
     at a distance of--->  1645

David Duncan House
     at a distance of--->  1643

Georgy Porgy's
     a

In [36]:
map_parkwoods=folium.Map(location=[43.7532586,-79.3296565],zoom_start=13)
map_parkwoods

In [37]:
for res in res2:
    latit=res['venue']['location']['lat']
    longit=res['venue']['location']['lng']
    label=res['venue']['name']
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [latit, longit],
        radius=5,
        popup=label,
        color='black',
        fill=True,
        fill_color='#36FEF8',
        fill_opacity=0.7,
        parse_html=False).add_to(map_parkwoods)
map_parkwoods

In [38]:
df_clustered=pd.DataFrame(columns=['venue','latitude','longitude','venue_type'])
for res in res2:
    latit=res['venue']['location']['lat']
    longit=res['venue']['location']['lng']
    label=res['venue']['name']
    name=res['venue']['categories'][0]['name']
    df_clustered=df_clustered.append({'venue':label,'latitude':latit,'longitude':longit,'venue_type':name},ignore_index=True)
df_clustered.head(9)

,venue,latitude,longitude,venue_type
0,Allwyn's Bakery,43.759840,-79.324719,Caribbean Restaurant
1,Donalda Golf & Country Club,43.752816,-79.342741,Golf Course
2,Brookbanks Park,43.751976,-79.332140,Park
3,Graydon Hall Manor,43.763923,-79.342961,Event Space
4,LCBO,43.757774,-79.314257,Liquor Store
5,Darband Restaurant,43.755194,-79.348498,Middle Eastern Restaurant
6,Island Foods,43.745866,-79.346035,Caribbean Restaurant
7,LA Fitness,43.747665,-79.347077,Gym / Fitness Center
8,Galleria Supermarket,43.753520,-79.349518,Supermarket


## Clustering Neighborhood

In [0]:
from sklearn.cluster import KMeans

In [40]:
kclusters=5
kmeans = KMeans(n_clusters=kclusters, random_state=0)
kmeans.fit(df_clustered[['latitude','longitude']])

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
       n_clusters=5, n_init=10, n_jobs=None, precompute_distances='auto',
       random_state=0, tol=0.0001, verbose=0)

In [0]:
df_clustered['cluster']=kmeans.labels_

In [42]:
df_clustered.head()

,venue,latitude,longitude,venue_type,cluster
0,Allwyn's Bakery,43.759840,-79.324719,Caribbean Restaurant,0
1,Donalda Golf & Country Club,43.752816,-79.342741,Golf Course,1
2,Brookbanks Park,43.751976,-79.332140,Park,0
3,Graydon Hall Manor,43.763923,-79.342961,Event Space,1
4,LCBO,43.757774,-79.314257,Liquor Store,3


In [0]:
import matplotlib.cm as cm
import matplotlib.colors as colors

In [45]:
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

map_clusters=folium.Map(location=[43.7532586,-79.3296565],zoom_start=13)

markers_colors = []
for lat, lon, poi, cluster in zip(df_clustered['latitude'], df_clustered['longitude'], df_clustered['venue'], df_clustered['cluster']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## Lets assume that the person wants to set up a Coffee Shop

In [46]:
cluster=df_clustered['cluster'].value_counts().index
count=df_clustered['cluster'].value_counts().values
btype_count=list()
for i in cluster:
    btype_count.append(sum(1 for p in ((df_clustered[df_clustered['cluster']==i]['venue_type']=='Coffee Shop').values) if p==True))
df_clustered_btype=pd.DataFrame({'cluster':cluster,'count':count,'btype_count':btype_count})
df_clustered_btype

,cluster,count,btype_count
0,1,31,2
1,3,22,3
2,2,22,3
3,0,12,2
4,4,6,0


### We can see that there are Coffee Shops located in all clusters except Cluster 4

In [47]:
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

map_clusters=folium.Map(location=[43.7532586,-79.3296565],zoom_start=14)

markers_colors = []
for lat, lon, poi, cluster,venue_type in zip(df_clustered['latitude'], df_clustered['longitude'], df_clustered['venue'], df_clustered['cluster'],df_clustered['venue_type']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    if venue_type=='Supermarket':
        folium.CircleMarker(
        [lat, lon],
        radius=7,
        popup=label,
        color='black',
        fill=True,
        fill_color='black',
        fill_opacity=0.9).add_to(map_clusters)
    else:
        folium.CircleMarker(
        [lat, lon],
        radius=2,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.4).add_to(map_clusters)
       
map_clusters

In [48]:
df_clustered_btype_fil=df_clustered_btype[df_clustered_btype['count']==df_clustered_btype['count'].max()]
df_clustered_btype_fil

,cluster,count,btype_count
0,1,31,2


In [49]:
kmeans.cluster_centers_[3]

array([ 43.75789069, -79.31299252])

# The best suitable cluster to start a Coffee Shop in "Parkwoods" is cluster 4

Week5 Capstone project :An extension to the toronto clustering and segmentation from week3
# Observations:
1. Objective:To find the best cluster to start a coffee shop in toronto
2.From the above results and maps it can be clearly noted that
3.Cluster 3 and 2 have a high count followed by cluster 0 and 1
4.Since cluster 4 has a zero bytype count it would be the best cluster to establish a coffee shop in "Parkwoods"



# Thankyou
# Saranesh ManiRatna.K